# Sub-Workflow Parallel Request Handling

## Overview

This notebook demonstrates **specialized parallel interception** - showing how multiple parent executors can handle different request types from the same sub-workflow using typed handlers.

### Resource Allocation System:

```
Parent Workflow (Two Specialized Interceptors):
  
  Coordinator (start)
      |
      | Mixed request list [ResourceRequest, PolicyCheckRequest]
      v
  [ Sub-Workflow: WorkflowExecutor(ResourceRequester) ]
      |
      | Emits different RequestInfoMessage types:
      |   - ResourceRequest (cpu, memory, disk)
      |   - PolicyCheckRequest (quota, compliance, security)
      v
  Parent routes by type:
      |
      |-- ResourceRequest → ResourceCache.handle_resource_request()
      |   |
      |   |-- Cache hit → Send RequestResponse(source="cache")
      |   |-- Cache miss → Forward to external
      |
      |-- PolicyCheckRequest → PolicyEngine.handle_policy_request()
          |
          |-- Within quota → Send RequestResponse(approved=True)
          |-- Exceeds quota → Forward to external
```

### Key Concepts:

1. **Type-Based Routing**: Different executors handle different message types
2. **Specialized Interceptors**: ResourceCache vs. PolicyEngine
3. **Parallel Processing**: Multiple request types processed concurrently
4. **Automatic Filtering**: Handlers only receive matching types
5. **Fallback Forwarding**: Unhandled requests go to external services
6. **Result Collection**: Both interceptors collect their results

### What You Learn:

- Create multiple RequestInfoMessage subclasses
- Build specialized interceptor executors
- Use type-based handler routing
- Handle mixed request types from one sub-workflow
- Implement cache and policy layers
- Coordinate multiple interception strategies

## Prerequisites

- Agent Framework installed: `pip install agent-framework`
- No external services required (simulated via RequestInfoExecutor)

## Setup and Imports

In [ ]:
import asyncio
from dataclasses import dataclass
from typing import Any

import os
from dotenv import load_dotenv
from agent_framework import (
    Executor,
    RequestInfoExecutor,
    RequestInfoMessage,
    RequestResponse,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowExecutor,
    handler,
)
from typing_extensions import Never
# Load environment variables from .env file
load_dotenv('../../.env')


## Define Domain-Specific Request Types

### Two Request Families:

1. **ResourceRequest**: Computing resources (CPU, memory, disk)
2. **PolicyCheckRequest**: Policy validation (quota, compliance, security)

Each type routes to different interceptors!

In [ ]:
@dataclass
class ResourceRequest(RequestInfoMessage):
    """Request for computing resources."""
    resource_type: str = "cpu"  # cpu, memory, disk, etc.
    amount: int = 1
    priority: str = "normal"  # low, normal, high


@dataclass
class PolicyCheckRequest(RequestInfoMessage):
    """Request to check resource allocation policy."""
    resource_type: str = ""
    amount: int = 0
    policy_type: str = "quota"  # quota, compliance, security


@dataclass
class ResourceResponse:
    """Response with allocated resources."""
    resource_type: str
    allocated: int
    source: str  # Which system provided the resources


@dataclass
class PolicyResponse:
    """Response from policy check."""
    approved: bool
    reason: str


@dataclass
class RequestFinished:
    """Signals sub-workflow completion."""
    pass


print("✓ Message types defined")

## Create Sub-Workflow Executor

### ResourceRequester

**Demonstrates:**
- Processing mixed request types
- Emitting different RequestInfoMessage subclasses
- Handling typed responses (ResourceResponse, PolicyResponse)
- Request counting for completion tracking

In [ ]:
class ResourceRequester(Executor):
    """Simple executor that requests resources and checks policies."""

    def __init__(self):
        super().__init__(id="resource_requester")
        self._request_count = 0

    @handler
    async def request_resources(
        self,
        requests: list[dict[str, Any]],
        ctx: WorkflowContext[ResourceRequest | PolicyCheckRequest],
    ) -> None:
        """Process a list of resource requests."""
        print(f"🏭 Sub-workflow processing {len(requests)} requests")
        self._request_count += len(requests)

        for req_data in requests:
            req_type = req_data.get("request_type", "resource")

            request: ResourceRequest | PolicyCheckRequest
            if req_type == "resource":
                print(f"  📦 Requesting resource: {req_data.get('type', 'cpu')} x{req_data.get('amount', 1)}")
                request = ResourceRequest(
                    resource_type=req_data.get("type", "cpu"),
                    amount=req_data.get("amount", 1),
                    priority=req_data.get("priority", "normal"),
                )
                # Send to parent workflow for interception - not to target_id
                await ctx.send_message(request)
            elif req_type == "policy":
                print(
                    f"  🛡️  Checking policy: {req_data.get('type', 'cpu')} x{req_data.get('amount', 1)} "
                    f"({req_data.get('policy_type', 'quota')})"
                )
                request = PolicyCheckRequest(
                    resource_type=req_data.get("type", "cpu"),
                    amount=req_data.get("amount", 1),
                    policy_type=req_data.get("policy_type", "quota"),
                )
                # Send to parent workflow for interception - not to target_id
                await ctx.send_message(request)

    @handler
    async def handle_resource_response(
        self,
        response: RequestResponse[ResourceRequest, ResourceResponse],
        ctx: WorkflowContext[Never, RequestFinished],
    ) -> None:
        """Handle resource allocation response."""
        if response.data:
            source_icon = "🏪" if response.data.source == "cache" else "🌐"
            print(
                f"📦 {source_icon} Sub-workflow received: {response.data.allocated} {response.data.resource_type} "
                f"from {response.data.source}"
            )
            if self._collect_results():
                # Yield completion result to the parent workflow
                await ctx.yield_output(RequestFinished())

    @handler
    async def handle_policy_response(
        self,
        response: RequestResponse[PolicyCheckRequest, PolicyResponse],
        ctx: WorkflowContext[Never, RequestFinished],
    ) -> None:
        """Handle policy check response."""
        if response.data:
            status_icon = "✅" if response.data.approved else "❌"
            print(
                f"🛡️  {status_icon} Sub-workflow received policy response: "
                f"{response.data.approved} - {response.data.reason}"
            )
            if self._collect_results():
                # Yield completion result to the parent workflow
                await ctx.yield_output(RequestFinished())

    def _collect_results(self) -> bool:
        """Collect and summarize results."""
        self._request_count -= 1
        print(f"📊 Sub-workflow completed request ({self._request_count} remaining)")
        return self._request_count == 0


print("✓ ResourceRequester executor created")

## Create Specialized Interceptor 1: ResourceCache

### ResourceCache

**Demonstrates:**
- Typed handler for ResourceRequest (ONLY)
- Cache hit/miss logic
- Local response creation
- External request forwarding
- Result collection from external responses

In [ ]:
class ResourceCache(Executor):
    """Interceptor that handles RESOURCE requests from cache using typed routing."""

    # Use class attributes to avoid Pydantic assignment restrictions
    cache: dict[str, int] = {"cpu": 10, "memory": 50, "disk": 100}
    results: list[ResourceResponse] = []

    def __init__(self):
        super().__init__(id="resource_cache")

    @handler
    async def handle_resource_request(
        self, 
        request: ResourceRequest, 
        ctx: WorkflowContext[RequestResponse[ResourceRequest, Any] | ResourceRequest]
    ) -> None:
        """Handle RESOURCE requests from sub-workflows and check cache first."""
        resource_request = request
        print(f"🏪 CACHE interceptor checking: {resource_request.amount} {resource_request.resource_type}")

        available = self.cache.get(resource_request.resource_type, 0)

        if available >= resource_request.amount:
            # We can satisfy from cache
            self.cache[resource_request.resource_type] -= resource_request.amount
            response_data = ResourceResponse(
                resource_type=resource_request.resource_type, 
                allocated=resource_request.amount, 
                source="cache"
            )
            print(f"  ✅ Cache satisfied: {resource_request.amount} {resource_request.resource_type}")
            self.results.append(response_data)

            # Send response back to sub-workflow
            response = RequestResponse(
                data=response_data, 
                original_request=request, 
                request_id=request.request_id
            )
            await ctx.send_message(response, target_id=request.source_executor_id)
        else:
            # Cache miss - forward to external
            print(f"  ❌ Cache miss: need {resource_request.amount}, have {available} {resource_request.resource_type}")
            await ctx.send_message(request)

    @handler
    async def collect_result(
        self, 
        response: RequestResponse[ResourceRequest, ResourceResponse], 
        ctx: WorkflowContext
    ) -> None:
        """Collect results from external requests that were forwarded."""
        if response.data and response.data.source != "cache":  # Don't double-count our own results
            self.results.append(response.data)
            print(
                f"🏪 🌐 Cache received external response: {response.data.allocated} {response.data.resource_type} "
                f"from {response.data.source}"
            )


print("✓ ResourceCache interceptor created")

## Create Specialized Interceptor 2: PolicyEngine

### PolicyEngine

**Demonstrates:**
- Typed handler for PolicyCheckRequest (ONLY)
- Quota validation logic
- Approval/rejection decisions
- Unknown policy type forwarding

In [ ]:
class PolicyEngine(Executor):
    """Interceptor that handles POLICY requests using typed routing."""

    # Use class attributes for simple sample state
    quota: dict[str, int] = {
        "cpu": 5,  # Only allow up to 5 CPU units
        "memory": 20,  # Only allow up to 20 memory units
        "disk": 1000,  # Liberal disk policy
    }
    results: list[PolicyResponse] = []

    def __init__(self):
        super().__init__(id="policy_engine")

    @handler
    async def handle_policy_request(
        self,
        request: PolicyCheckRequest,
        ctx: WorkflowContext[RequestResponse[PolicyCheckRequest, Any] | PolicyCheckRequest],
    ) -> None:
        """Handle POLICY requests from sub-workflows and apply rules."""
        policy_request = request
        print(
            f"🛡️  POLICY interceptor checking: {policy_request.amount} {policy_request.resource_type}, "
            f"policy={policy_request.policy_type}"
        )

        quota_limit = self.quota.get(policy_request.resource_type, 0)

        if policy_request.policy_type == "quota":
            if policy_request.amount <= quota_limit:
                response_data = PolicyResponse(
                    approved=True, 
                    reason=f"Within quota ({quota_limit})"
                )
                print(f"  ✅ Policy approved: {policy_request.amount} <= {quota_limit}")
                self.results.append(response_data)

                # Send response back to sub-workflow
                response = RequestResponse(
                    data=response_data, 
                    original_request=request, 
                    request_id=request.request_id
                )
                await ctx.send_message(response, target_id=request.source_executor_id)
                return

            # Exceeds quota - forward to external for review
            print(f"  ❌ Policy exceeds quota: {policy_request.amount} > {quota_limit}, forwarding to external")
            await ctx.send_message(request)
            return

        # Unknown policy type - forward to external
        print(f"  ❓ Unknown policy type: {policy_request.policy_type}, forwarding")
        await ctx.send_message(request)

    @handler
    async def collect_policy_result(
        self, 
        response: RequestResponse[PolicyCheckRequest, PolicyResponse], 
        ctx: WorkflowContext
    ) -> None:
        """Collect policy results from external requests that were forwarded."""
        if response.data:
            self.results.append(response.data)
            print(f"🛡️  🌐 Policy received external response: {response.data.approved} - {response.data.reason}")


print("✓ PolicyEngine interceptor created")

## Create Coordinator Executor

In [ ]:
class Coordinator(Executor):
    """Coordinates the workflow execution."""
    
    def __init__(self):
        super().__init__(id="coordinator")

    @handler
    async def start(
        self, 
        requests: list[dict[str, Any]], 
        ctx: WorkflowContext[list[dict[str, Any]]]
    ) -> None:
        """Start the resource allocation process."""
        await ctx.send_message(requests, target_id="resource_workflow")

    @handler
    async def handle_completion(
        self, 
        completion: RequestFinished, 
        ctx: WorkflowContext
    ) -> None:
        """Handle sub-workflow completion (from yielded output)."""
        print("🎯 Main workflow received completion.")


print("✓ Coordinator executor created")

## Build Workflows

### Sub-Workflow Graph:

```
ResourceRequester (start)
    ↓
RequestInfoExecutor ("sub_request_info")
    ↓
ResourceRequester (handle responses)
```

### Parent Workflow Graph (Type-Based Routing):

```
Coordinator (start)
    ↓
WorkflowExecutor (sub-workflow)
    ↓ (ResourceRequest)
ResourceCache.handle_resource_request()
    ↓ (PolicyCheckRequest)
PolicyEngine.handle_policy_request()
    ↓ (RequestResponse)
Back to WorkflowExecutor
    ↓ (Forwarded requests)
RequestInfoExecutor ("main_request_info")
```

In [ ]:
print("🚀 Starting Sub-Workflow Parallel Request Interception Demo...")
print("=" * 60)

# Create the sub-workflow
resource_requester = ResourceRequester()
sub_request_info = RequestInfoExecutor(id="sub_request_info")

sub_workflow = (
    WorkflowBuilder()
    .set_start_executor(resource_requester)
    .add_edge(resource_requester, sub_request_info)
    .add_edge(sub_request_info, resource_requester)
    .build()
)

print("✓ Sub-workflow created")

# Create parent workflow with PROPER interceptor pattern
cache = ResourceCache()  # Intercepts ResourceRequest
policy = PolicyEngine()  # Intercepts PolicyCheckRequest (different type!)
workflow_executor = WorkflowExecutor(sub_workflow, id="resource_workflow")
main_request_info = RequestInfoExecutor(id="main_request_info")

# Create a simple coordinator that starts the process
coordinator = Coordinator()

# TYPED ROUTING: Each executor handles specific typed RequestInfoMessage messages
main_workflow = (
    WorkflowBuilder()
    .set_start_executor(coordinator)
    .add_edge(coordinator, workflow_executor)  # Start sub-workflow
    .add_edge(workflow_executor, coordinator)  # Sub-workflow completion back to coordinator
    .add_edge(workflow_executor, cache)  # WorkflowExecutor sends ResourceRequest to cache
    .add_edge(workflow_executor, policy)  # WorkflowExecutor sends PolicyCheckRequest to policy
    .add_edge(cache, workflow_executor)  # Cache sends RequestResponse back
    .add_edge(policy, workflow_executor)  # Policy sends RequestResponse back
    .add_edge(cache, main_request_info)  # Cache forwards ResourceRequest to external
    .add_edge(policy, main_request_info)  # Policy forwards PolicyCheckRequest to external
    .add_edge(main_request_info, workflow_executor)  # External responses back to sub-workflow
    .build()
)

print("✓ Parent workflow created with specialized interceptors")

## Prepare Test Data

Testing with 7 mixed requests:
1. **Resource: cpu x2** - Cache hit (10 available)
2. **Policy: cpu x3, quota** - Policy hit (limit 5)
3. **Resource: memory x15** - Cache hit (50 available)
4. **Policy: memory x100, quota** - Policy miss (limit 20) → External
5. **Resource: gpu x1** - Cache miss (not in cache) → External
6. **Policy: disk x500, quota** - Policy hit (limit 1000)
7. **Policy: cpu x1, security** - Unknown policy type → External

In [ ]:
test_requests = [
    {"request_type": "resource", "type": "cpu", "amount": 2, "priority": "normal"},  # Cache hit
    {"request_type": "policy", "type": "cpu", "amount": 3, "policy_type": "quota"},  # Policy hit
    {"request_type": "resource", "type": "memory", "amount": 15, "priority": "normal"},  # Cache hit
    {"request_type": "policy", "type": "memory", "amount": 100, "policy_type": "quota"},  # Policy miss → external
    {"request_type": "resource", "type": "gpu", "amount": 1, "priority": "high"},  # Cache miss → external
    {"request_type": "policy", "type": "disk", "amount": 500, "policy_type": "quota"},  # Policy hit
    {"request_type": "policy", "type": "cpu", "amount": 1, "policy_type": "security"},  # Unknown policy → external
]

print(f"\n🧪 Testing with {len(test_requests)} mixed requests:")
for i, req in enumerate(test_requests, 1):
    req_icon = "📦" if req["request_type"] == "resource" else "🛡️"
    print(
        f"  {i}. {req_icon} {req['type']} x{req['amount']} "
        f"({req.get('priority', req.get('policy_type', 'default'))})"
    )
print("=" * 70)

## Run Workflow

Watch as:
- ResourceRequest messages → ResourceCache interceptor
- PolicyCheckRequest messages → PolicyEngine interceptor
- Each interceptor decides: handle locally or forward
- Responses routed back to sub-workflow automatically

In [ ]:
print("\n🎬 Running workflow...")
events = await main_workflow.run(test_requests)

## Handle External Requests

Requests that couldn't be handled by either interceptor are collected and simulated.

In [ ]:
request_events = events.get_request_info_events()

if request_events:
    print(f"\n🌐 Handling {len(request_events)} external request(s)...")

    external_responses: dict[str, Any] = {}
    for event in request_events:
        if isinstance(event.data, ResourceRequest):
            # Handle ResourceRequest - create ResourceResponse
            resource_response = ResourceResponse(
                resource_type=event.data.resource_type, 
                allocated=event.data.amount, 
                source="external_provider"
            )
            external_responses[event.request_id] = resource_response
            print(f"  🏭 External provider: {resource_response.allocated} {resource_response.resource_type}")
        elif isinstance(event.data, PolicyCheckRequest):
            # Handle PolicyCheckRequest - create PolicyResponse
            policy_response = PolicyResponse(
                approved=True, 
                reason="External policy service approved"
            )
            external_responses[event.request_id] = policy_response
            print(f"  🔒 External policy: {'✅ APPROVED' if policy_response.approved else '❌ DENIED'}")

    await main_workflow.send_responses(external_responses)
else:
    print("\n🎯 All requests were intercepted internally!")

## Display Results Analysis

In [ ]:
print("\n" + "=" * 70)
print("📊 RESULTS ANALYSIS")
print("=" * 70)

print(f"\n🏪 Cache Results ({len(cache.results)} handled):")
for result in cache.results:
    print(f"  ✅ {result.allocated} {result.resource_type} from {result.source}")

print(f"\n🛡️  Policy Results ({len(policy.results)} handled):")
for result in policy.results:
    status_icon = "✅" if result.approved else "❌"
    print(f"  {status_icon} Approved: {result.approved} - {result.reason}")

print("\n💾 Final Cache State:")
for resource, amount in cache.cache.items():
    print(f"  📦 {resource}: {amount} remaining")

print("\n📈 Summary:")
print(f"  🎯 Total requests: {len(test_requests)}")
print(f"  🏪 Resource requests handled: {len(cache.results)}")
print(f"  🛡️  Policy requests handled: {len(policy.results)}")
print(f"  🌐 External requests: {len(request_events) if request_events else 0}")

print("\n" + "=" * 70)

## Expected Output Pattern

```
🚀 Starting Sub-Workflow Parallel Request Interception Demo...
============================================================
✓ Sub-workflow created
✓ Parent workflow created with specialized interceptors

🧪 Testing with 7 mixed requests:
  1. 📦 cpu x2 (normal)
  2. 🛡️ cpu x3 (quota)
  3. 📦 memory x15 (normal)
  4. 🛡️ memory x100 (quota)
  5. 📦 gpu x1 (high)
  6. 🛡️ disk x500 (quota)
  7. 🛡️ cpu x1 (security)
======================================================================

🎬 Running workflow...
🏭 Sub-workflow processing 7 requests
  📦 Requesting resource: cpu x2
  🛡️  Checking policy: cpu x3 (quota)
  📦 Requesting resource: memory x15
  🛡️  Checking policy: memory x100 (quota)
  📦 Requesting resource: gpu x1
  🛡️  Checking policy: disk x500 (quota)
  🛡️  Checking policy: cpu x1 (security)
🏪 CACHE interceptor checking: 2 cpu
  ✅ Cache satisfied: 2 cpu
📦 🏪 Sub-workflow received: 2 cpu from cache
📊 Sub-workflow completed request (6 remaining)
🛡️  POLICY interceptor checking: 3 cpu, policy=quota
  ✅ Policy approved: 3 <= 5
🛡️  ✅ Sub-workflow received policy response: True - Within quota (5)
📊 Sub-workflow completed request (5 remaining)
🏪 CACHE interceptor checking: 15 memory
  ✅ Cache satisfied: 15 memory
📦 🏪 Sub-workflow received: 15 memory from cache
📊 Sub-workflow completed request (4 remaining)
🛡️  POLICY interceptor checking: 100 memory, policy=quota
  ❌ Policy exceeds quota: 100 > 20, forwarding to external
🏪 CACHE interceptor checking: 1 gpu
  ❌ Cache miss: need 1, have 0 gpu
🛡️  POLICY interceptor checking: 500 disk, policy=quota
  ✅ Policy approved: 500 <= 1000
🛡️  ✅ Sub-workflow received policy response: True - Within quota (1000)
📊 Sub-workflow completed request (3 remaining)
🛡️  POLICY interceptor checking: 1 cpu, policy=security
  ❓ Unknown policy type: security, forwarding

🌐 Handling 3 external request(s)...
  🏭 External provider: 1 gpu
  🔒 External policy: ✅ APPROVED
  🔒 External policy: ✅ APPROVED
🏪 🌐 Cache received external response: 1 gpu from external_provider
📦 🌐 Sub-workflow received: 1 gpu from external_provider
📊 Sub-workflow completed request (2 remaining)
🛡️  🌐 Policy received external response: True - External policy service approved
🛡️  ✅ Sub-workflow received policy response: True - External policy service approved
📊 Sub-workflow completed request (1 remaining)
🛡️  🌐 Policy received external response: True - External policy service approved
🛡️  ✅ Sub-workflow received policy response: True - External policy service approved
📊 Sub-workflow completed request (0 remaining)
🎯 Main workflow received completion.

======================================================================
📊 RESULTS ANALYSIS
======================================================================

🏪 Cache Results (3 handled):
  ✅ 2 cpu from cache
  ✅ 15 memory from cache
  ✅ 1 gpu from external_provider

🛡️  Policy Results (4 handled):
  ✅ Approved: True - Within quota (5)
  ✅ Approved: True - Within quota (1000)
  ✅ Approved: True - External policy service approved
  ✅ Approved: True - External policy service approved

💾 Final Cache State:
  📦 cpu: 8 remaining
  📦 memory: 35 remaining
  📦 disk: 100 remaining

📈 Summary:
  🎯 Total requests: 7
  🏪 Resource requests handled: 3
  🛡️  Policy requests handled: 4
  🌐 External requests: 3

======================================================================
```

## Key Takeaways

### 1. Type-Based Routing

```python
# Each executor handles specific message type
class ResourceCache(Executor):
    @handler
    async def handle_resource_request(
        self,
        request: ResourceRequest,  # ONLY ResourceRequest
        ctx: WorkflowContext[...]
    ) -> None:
        ...

class PolicyEngine(Executor):
    @handler
    async def handle_policy_request(
        self,
        request: PolicyCheckRequest,  # ONLY PolicyCheckRequest
        ctx: WorkflowContext[...]
    ) -> None:
        ...
```

**Framework Behavior:**
- Inspects message type at runtime
- Routes to matching handler signature
- Both executors receive messages from same source
- No manual routing code needed

### 2. Multiple Interceptor Pattern

```python
# In workflow graph
builder = (
    WorkflowBuilder()
    .add_edge(workflow_executor, cache)    # ResourceRequest → cache
    .add_edge(workflow_executor, policy)   # PolicyCheckRequest → policy
    .add_edge(cache, workflow_executor)    # Responses back
    .add_edge(policy, workflow_executor)   # Responses back
)
```

**Benefits:**
- Separation of concerns
- Independent testing
- Easy to add new interceptors
- Clear responsibility boundaries

### 3. Specialized Interception Logic

#### Resource Cache Pattern
```python
if available >= requested:
    # Handle locally from cache
    self.cache[resource] -= requested
    response = RequestResponse(data=result, ...)
    await ctx.send_message(response, target_id=source_executor_id)
else:
    # Forward to external provider
    await ctx.send_message(request)
```

#### Policy Engine Pattern
```python
if policy_type == "quota":
    if amount <= quota_limit:
        # Approve locally
        response = RequestResponse(data=PolicyResponse(approved=True), ...)
        await ctx.send_message(response, target_id=source_executor_id)
    else:
        # Forward for manual review
        await ctx.send_message(request)
else:
    # Unknown policy - forward to specialized service
    await ctx.send_message(request)
```

### 4. Result Collection Patterns

```python
class Interceptor(Executor):
    results: list[Response] = []
    
    @handler
    async def handle_request(self, request: Request, ctx) -> None:
        if can_handle_locally:
            result = process(request)
            self.results.append(result)  # Collect local result
            ...
    
    @handler
    async def collect_external_result(self, response: RequestResponse, ctx) -> None:
        # Collect results from forwarded requests
        if response.data and is_external(response.data):
            self.results.append(response.data)
```

### 5. Concurrent Request Handling

**Sub-workflow perspective:**
```python
# Send 7 requests concurrently - no await between sends
for request_data in requests:
    request = create_request(request_data)
    await ctx.send_message(request)  # Non-blocking
```

**Interceptor perspective:**
```python
# Each interceptor handles its type independently
# No coordination needed between ResourceCache and PolicyEngine
# Framework ensures correct correlation via request_id
```

### 6. Production Patterns

#### Multi-Tier Cache
```python
class L1Cache(Executor):
    @handler
    async def check_l1(self, request: DataRequest, ctx) -> None:
        if request.key in self.memory_cache:
            # L1 hit - fastest
            ...
        else:
            # Forward to L2
            await ctx.send_message(request)

class L2Cache(Executor):
    @handler
    async def check_l2(self, request: DataRequest, ctx) -> None:
        if request.key in self.disk_cache:
            # L2 hit - slower but available
            ...
        else:
            # Forward to external
            await ctx.send_message(request)
```

#### Priority-Based Routing
```python
class PriorityRouter(Executor):
    @handler
    async def route_request(self, request: ResourceRequest, ctx) -> None:
        if request.priority == "high":
            # High priority - allocate immediately
            response = RequestResponse(data=allocate_premium(request), ...)
            await ctx.send_message(response, target_id=request.source_executor_id)
        elif request.priority == "normal":
            # Normal - check availability
            await ctx.send_message(request)  # Forward to cache
        else:
            # Low priority - queue for batch processing
            self.queue.append(request)
```

#### A/B Testing
```python
class ABTestInterceptor(Executor):
    @handler
    async def ab_test(self, request: APIRequest, ctx) -> None:
        if hash(request.user_id) % 2 == 0:
            # Group A - use optimized path
            result = optimized_handler(request)
            response = RequestResponse(data=result, ...)
            await ctx.send_message(response, target_id=request.source_executor_id)
        else:
            # Group B - use standard path
            await ctx.send_message(request)
```

### 7. Error Handling in Interceptors

```python
@handler
async def handle_request(self, request: Request, ctx) -> None:
    try:
        if can_handle:
            result = process(request)
            response = RequestResponse(data=result, ...)
            await ctx.send_message(response, target_id=request.source_executor_id)
        else:
            await ctx.send_message(request)
    except Exception as e:
        # Send error response instead of crashing
        error_response = RequestResponse(
            data=None,
            error=str(e),
            original_request=request,
            request_id=request.request_id
        )
        await ctx.send_message(error_response, target_id=request.source_executor_id)
```